In [1]:
import seaborn as sns

tips = sns.load_dataset('tips')

In [3]:
import pandas as pd
import numpy as np

In [5]:
tips_OH=pd.get_dummies(tips)

In [8]:
def DeOH_column(df,column):
    """Tries to find all the columns of the original dataframe, os numerical+De_OH_encoded"""
    cols=df.columns
    i=list(cols).index(column)
    future_colnames=[]
  
    pos=findall("_",column)       
    if pos==[-1]:       
        #future_colnames.append(column) We don´t do this to avoid the ones that haven´t been OH encoded       
        return column       

    names=[column[:p] for p in pos]       
    name_final=""       
    for name in names:       
        possible_columns=[]       
        long=len(name)       
        for j in range(len(cols)):       
            if j==i:       
                continue       
            if name==cols[j][:long]:       
                #If the column isn´t actually followed by a _ we don´t want it (if it´s the exact length it might be the original column (which shouldn´t be there anyways))       
                if len(cols[j])>long:       
                    if cols[j][long]!="_":       
                        continue       
                possible_columns.append(cols[j])       
        if len(possible_columns)>=1:       
            name_final=name       
        if name_final not in future_colnames and name_final!="":       
            future_colnames.append(name_final)       
 
    if len(future_colnames)>1:
        print("column",column,"future_colnames",future_colnames)
        raise(Exception("Error, there should only be one column that matches the name of the interaction"))
   
    if len(future_colnames)==0:
        if len(names)==1:
            return names[0]
        else:
            print("column",column,"possible_names",names)
            raise(Exception("Error, there´s nothing to match with and it has multiple '_'"))          
 
    return future_colnames[0]

In [22]:
def Detect_OHcols(columns_df):
    future_colnames=[]
    for i in range(len(columns_df)):
        pos=findall("_",columns_df[i])
        if pos==[-1]:
            future_colnames.append(columns_df[i])
            continue
        
        names=[columns_df[i][:p] for p in pos]
        #We´re gonna go over all the possibilities, so each round we´ve got a longer name
        #We are gonna assume that the longest possible name is the correct name
        name_final=""
        for name in names:
            Has_a_match=False#possible_columns=[]
            long=len(name)
            for j in range(len(columns_df)):
                #We can´t check ourselves because we would just return our own OH encoded name
                if j==i:       
                    continue 
                if name==columns_df[j][:long]:
                    #If the column isn´t actually followed by a _ we don´t want it (we´re looking for other OH encoded like us, which if it exists it´ll be followed by a _)       
                    if len(columns_df[j])>long:       
                        if columns_df[j][long]!="_":       
                            continue  
                    Has_a_match=True#possible_columns.append(columns_df[j])
            if Has_a_match:#len(possible_columns)>=1:
                name_final=name
            #It´s possible that we´re alone as it´s a binary and drop first-> we take the longest name
        if name_final not in future_colnames and name_final!="":
            future_colnames.append(name_final)
        #It´s possible that we don´t find a match, and most likely it´s because we´re dealing with a numerical (not OH)
        if name_final=="" and columns_df[i] not in future_colnames:
            future_colnames.append(columns_df[i])


    return future_colnames

In [9]:
def DeOneHotEncodeColumn(df,colname):
    """This function decodes onehotencoded columns that start with colname (and eliminates those columns)"""

    #These are the columns we think are onehotencoded
    cols_tryout=[col for col in df.columns if colname+"_"==col[:len(colname)+1]]
    #The cols from cols_tryout aren´t necessarily binary, so we´ll check
    cols=[]
    for col in cols_tryout:
        EsBin=True
        i=0
        #This other version I think could be faster when there are only ones and 0s faster (python faster on vector ops)
        #unicos=df[col].unique()
        #if len(unicos)<=2:
        #    for a in unicos:
        #        if a not in [0,1]:
        #            EsBin=False

        while EsBin and i<df.shape[0]:
            if df[col][i]!=0 and df[col][i]!=1:
                EsBin=False
            i+=1
        if EsBin:
            cols.append(col)
    #If there aren´t at least 2 binary columns... what are u doing lol
    if len(cols)<2:
        return df
    sol=np.array(["" for i in range(df.shape[0])],dtype="object")
    #Let´s get the output of our decoded column ()
    for col in cols:
        sol+=np.array([col[len(colname)+1:] if x else "" for x in df[col]],dtype="object")

    #Let´s drop the columns we used and put the decoded column in place :)
    df[colname]=sol
    df=df.drop(cols,axis=1)
    return df

In [10]:
#Finds all the positions where char is in word
def findall(char,word):
    pos=[word.find(char)]
    suma=pos[0]+1
    while pos[-1]!=-1:
        pos.append(word[suma:].find(char))
        suma+=pos[-1]+1#(hay que sumar 1 por el "_" que hay que borrar)
    if pos[0]==-1:
        return pos
    pos=pos[:-1]
    suma=pos[0]
    for i in range(1,len(pos)):
        suma+=pos[i]+1#Hay que sumar 1 por los arrays empiezan en 0)
        pos[i]=suma
    return pos

In [11]:
#This function detects all the possible columns that have been one hot encoded, it does so by checking for _
#This is because when creating a OH of "EVENT" we get columns such as EVENT_SPAIN, EVENT_FRANCE....
#Since the column we´re looking for is EVENT_TYPE, we´re gonna make sure to be able to get the longest string that ends in _
#And that is at the beginning of multiple column names
def Detect_OHcols(df):
    cols=df.columns
    future_colnames=[]
    for i in range(len(cols)):
        pos=findall("_",cols[i])
        names=[cols[i][:p] for p in pos]
        name_final=""
        for name in names:
            possible_columns=[]
            long=len(name)
            for j in range(i+1,len(cols)):
                if name==cols[j][:long]:
                    possible_columns.append(cols[j])
            if len(possible_columns)>=1:
                name_final=name
            if name_final not in future_colnames and name_final!="":
                future_colnames.append(name_final)

    return future_colnames

In [12]:
#We´re gonna detect the columns, sort their names so that EVENT_TYPE is done before EVENT...and done :)
def DeOneHotEncode(df,exclude=[]):
    cols=pd.Series(Detect_OHcols(df))
    cols=cols.sort_values(ascending=False)
    for col in cols:
        if col not in exclude:
            df=DeOneHotEncodeColumn(df,col)
    return df

In [23]:
Detect_OHcols(tips_OH.columns)

['total_bill', 'tip', 'size', 'sex', 'smoker', 'day', 'time']

In [24]:
tips.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')